In [1]:
from influxdb.commands import InfluxDBQueries, DataProcessor, LSTMModel
import warnings
import pandas as pd
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [2]:
df_train = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2023-06-01T01:00:00Z",
                                                stop="2025-01-01T01:00:00Z")
df_val = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-01-01T01:00:00Z",
                                              stop="2025-03-01T01:00:00Z")
df_test = InfluxDBQueries.get_data_from_influx(groupby_time='5m', pair='BTCUSDT', start="2025-03-01T01:00:00Z",
                                               stop="2025-04-20T01:00:00Z")

In [3]:
df_combined = pd.concat([df_train, df_test, df_val])

In [4]:
df_combined, scaler, columns, sclaer_y = DataProcessor.add_features(df_combined, classification=True, group_time='5m')

In [5]:
df_combined.dropna(inplace=True)

In [6]:
columns, boruta = DataProcessor.select_features_boruta(df_combined, columns, classification=True)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	43
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	43
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	43
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	43
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	43
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	43
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	43
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	27
Iteration: 	9 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	27
Iteration: 	10 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	27
Iteration: 	11 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	27
Iteration: 	12 / 100
Confirmed: 	9
Tentative: 	7
Rejected: 	27
Iteration: 	13 / 100
Confirmed: 	9
Tentative: 	6
Rejected: 	28
Iteration: 	14 / 100
Confirmed: 	9
Tentative: 	6
Rejected: 	28
Iteration: 	15 / 100
Confirmed: 	9
Tentative: 	6
Rejected: 	28
Iteration: 	16 / 100
Confirmed: 	9
Tentative: 	6
Rejected: 	28
I

In [7]:
df_train = df_combined.loc[df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]
df_val = df_combined.loc[
    (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_val.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')) &
    (df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') < df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ'))]
df_test = df_combined.loc[
    df_combined.index.strftime('%Y-%m-%dT%H:%M:%SZ') >= df_test.index[0].strftime('%Y-%m-%dT%H:%M:%SZ')]

In [8]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_val = df_val.dropna()

In [9]:
X_train, y_train = DataProcessor.make_sequences(df_train, columns_to_select=columns, window_size=288,forecast_horizon=1, classification=True)

In [10]:
X_val, y_val = DataProcessor.make_sequences(df_val, columns_to_select=columns, window_size=288, forecast_horizon=1, classification=True)

In [11]:
lstm_forecaster = LSTMModel(input_shape=(288, len(columns)), output_steps=1, classification=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 288, 64)        │         2,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 288, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling1d               │ (None, 288, 64)        │             0 │
│ (AveragePooling1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 288, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 288, 128)       │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 288, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 288, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 80)             │        66,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 80)             │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 169,617 (662.57 KB)

 Trainable params: 169,073 (660.44 KB)

 Non-trainable params: 544 (2.12 KB)

In [12]:
lstm_forecaster.fit(X_train, y_train, X_val, y_val, epochs=300, batch_size=128, patience=20)

Training started with EarlyStopping (patience=20)...
Epoch 1/300
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5052 - auc: 0.5061 - loss: 0.7899
Epoch 1: val_loss improved from inf to 0.69303, saving model to models/best_model.keras
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 1494s 1s/step - accuracy: 0.5052 - auc: 0.5061 - loss: 0.7899 - val_accuracy: 0.5133 - val_auc: 0.5158 - val_loss: 0.6930 - learning_rate: 0.0010
Epoch 2/300
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5106 - auc: 0.5145 - loss: 0.6934
Epoch 2: val_loss did not improve from 0.69303
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 1508s 1s/step - accuracy: 0.5106 - auc: 0.5145 - loss: 0.6934 - val_accuracy: 0.5011 - val_auc: 0.5063 - val_loss: 0.6939 - learning_rate: 0.0010
Epoch 3/300
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5126 - auc: 0.5173 - loss: 0.6933
Epoch 3: val_loss did not improve from 0.69303
1299/1299 ━━━━━━━━━━━━━━━━━━━━ 1522s 1s/step - accuracy: 0.5126 - auc: 0.5173 - loss: 0.6933 - val_accura

In [13]:
X_test, y_test = DataProcessor.make_sequences(df_test, columns_to_select=columns, window_size=288, forecast_horizon=1, classification=True)

In [14]:
lstm_forecaster.evaluate(X_test, y_test)

441/441 ━━━━━━━━━━━━━━━━━━━━ 41s 94ms/step - accuracy: 0.5162 - auc: 0.5278 - loss: 0.6919
Test Loss: 0.6918755769729614, MSE: 0.5143140554428101, Directional Accuracy: <function directional_accuracy at 0x139c2e980>
441/441 ━━━━━━━━━━━━━━━━━━━━ 42s 94ms/step
Confidence-Filtered Accuracy: 0.50, Coverage: 100.00%


0.6918755769729614